In [30]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

import pyspark

conf = SparkConf().setAppName('myApp3').setMaster('local')
sc = SparkContext(conf=conf)
spark=SparkSession(sc)


In [31]:
emp_df=spark.read.format('csv').option('header','true').option('interSchema','true').load('C:\\Users\\daesi\\Downloads\\emp.csv')
emp_df.printSchema()

root
 |-- empno: string (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: string (nullable = true)
 |-- hiredate: string (nullable = true)
 |-- sal: string (nullable = true)
 |-- comm: string (nullable = true)
 |-- deptno: string (nullable = true)



In [32]:
dept_df = spark.read.format('csv').option('header','true').option('interSchema','true').load('C:\\Users\\daesi\\Downloads\\dept.csv')
dept_df.printSchema()

root
 |-- deptno: string (nullable = true)
 |-- dname: string (nullable = true)
 |-- loc: string (nullable = true)



In [33]:
dept_df.show()

+------+----------+--------+
|deptno|     dname|     loc|
+------+----------+--------+
|    10|ACCOUNTING|NEW YORK|
|    20|  RESEARCH|  DALLAS|
|    30|     SALES| CHICAGO|
|    40|OPERATIONS|  BOSTON|
+------+----------+--------+



In [34]:
emp_df.createOrReplaceTempView('dfEmp_view')
dept_df.createOrReplaceTempView('dfDept_view')

In [35]:
query='''
    select *
    from dfEmp_view
'''
spark.sql(query).show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|null|    20|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|null|    30|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|null|    20|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|
| 9292|  JACK|

In [36]:
#쿼리로 조인
query='''
    select *
    from dfEmp_view,dfDept_view
    where dfEmp_view.deptno = dfDept_view.deptno
'''
viewEmpDeptJoin = spark.sql(query)
viewEmpDeptJoin.show()

+-----+------+---------+----+----------+----+----+------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|    20|  RESEARCH|  DALLAS|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|    30|     SALES| CHICAGO|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|    30|     SALES| CHICAGO|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|    20|  RESEARCH|  DALLAS|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|    30|     SALES| CHICAGO|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|    30|     SALES| CHICAGO|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|    10|ACCOUNTING|NEW YORK|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|    20|  RESEARCH|  DALLAS|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|    10|A

In [38]:
#데이터프레임 조인
dfJoinEmp = emp_df.join(dept_df,emp_df['deptno']==dept_df['deptno'])
dfJoinEmp.show()

+-----+------+---------+----+----------+----+----+------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|    20|  RESEARCH|  DALLAS|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|    30|     SALES| CHICAGO|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|    30|     SALES| CHICAGO|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|    20|  RESEARCH|  DALLAS|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|    30|     SALES| CHICAGO|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|    30|     SALES| CHICAGO|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|    10|ACCOUNTING|NEW YORK|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|    20|  RESEARCH|  DALLAS|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|    10|A

In [39]:
viewEmpDeptJoin.createOrReplaceTempView('empDeptJoin_view')

query = '''
    select*
    from empDeptJoin_view
    where loc="NEW YORK"
'''

spark.sql(query).show()

+-----+------+---------+----+----------+----+----+------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|    10|ACCOUNTING|NEW YORK|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|    10|ACCOUNTING|NEW YORK|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|    10|ACCOUNTING|NEW YORK|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+



In [40]:
query1 = '''
    select *
    from dfEmp_view
    where deptno = (
        select deptno
        from dfDept_view
        where loc='NEW YORK'
    
    )

'''
spark.sql(query1).show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|
+-----+------+---------+----+----------+----+----+------+



In [46]:
spark.sql(query1).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Filter (isnotnull(deptno#661) AND (deptno#661 = Subquery subquery#1110, [id=#1111]))
   :  +- Subquery subquery#1110, [id=#1111]
   :     +- AdaptiveSparkPlan isFinalPlan=false
   :        +- Project [deptno#686]
   :           +- Filter (isnotnull(loc#688) AND (loc#688 = NEW YORK))
   :              +- FileScan csv [deptno#686,loc#688] Batched: false, DataFilters: [isnotnull(loc#688), (loc#688 = NEW YORK)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/daesi/Downloads/dept.csv], PartitionFilters: [], PushedFilters: [IsNotNull(loc), EqualTo(loc,NEW YORK)], ReadSchema: struct<deptno:string,loc:string>
   +- FileScan csv [empno#654,ename#655,job#656,mgr#657,hiredate#658,sal#659,comm#660,deptno#661] Batched: false, DataFilters: [isnotnull(deptno#661)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/daesi/Downloads/emp.csv], PartitionFilters: [], PushedFilters: [IsNotNull(deptno)], ReadSchema: s

In [47]:
spark.sql(query).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Filter (isnotnull(sum(sal)#1120) AND (sum(sal)#1120 > 10.0))
   +- HashAggregate(keys=[loc#688], functions=[sum(cast(sal#659 as double))])
      +- Exchange hashpartitioning(loc#688, 200), ENSURE_REQUIREMENTS, [id=#1160]
         +- HashAggregate(keys=[loc#688], functions=[partial_sum(cast(sal#659 as double))])
            +- Project [sal#659, loc#688]
               +- BroadcastHashJoin [deptno#661], [deptno#686], Inner, BuildRight, false
                  :- Filter isnotnull(deptno#661)
                  :  +- FileScan csv [sal#659,deptno#661] Batched: false, DataFilters: [isnotnull(deptno#661)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/daesi/Downloads/emp.csv], PartitionFilters: [], PushedFilters: [IsNotNull(deptno)], ReadSchema: struct<sal:string,deptno:string>
                  +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, false]),false), [id=#1155]
                     +- F

In [43]:
query='''
    select loc, sum(sal)
    from empDeptjoin_view
    group by loc
'''
spark.sql(query).show()

+--------+--------+
|     loc|sum(sal)|
+--------+--------+
|  DALLAS| 10875.0|
| CHICAGO|  9400.0|
|NEW YORK|  8750.0|
+--------+--------+



In [44]:
query='''
    select loc, sum(sal)
    from empDeptjoin_view
    where loc like 'N%'
    group by loc
    having sum(sal)>10
'''
spark.sql(query).show()

+--------+--------+
|     loc|sum(sal)|
+--------+--------+
|NEW YORK|  8750.0|
+--------+--------+



In [48]:
sc.stop()

In [49]:
spark.stop()